In [1]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from tqdm.notebook import tqdm


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [ ]:
  !pip install optuna

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/project9/train_transformed.csv')
train_labels = pd.read_csv('/content/drive/MyDrive/project9/music30s_trainlabel.csv')
test =pd.read_csv('/content/drive/MyDrive/project9/test_transformed.csv')


In [ ]:
train_labels =train_labels.iloc[:,1:]
train_labels.head(3)


In [ ]:
display(train.head(3))
display(test.head(3))

In [ ]:
cols = [c for c in train.columns]

In [ ]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
train[cols] = stdsc.fit_transform(train[cols])
test[cols] = stdsc.transform(test[cols])

In [ ]:
display(train.head(3))
display(test.head(3))

In [ ]:
train = pd.concat([train,train_labels],axis=1)

In [ ]:
train.head()

In [ ]:
X = train.drop('label',axis=1)
y=train['label']

In [ ]:
import sklearn

In [ ]:
from sklearn.decomposition import PCA


In [ ]:
#95% of variance?
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.95)
X_new = pca.fit_transform(X)
test_new = pca.transform(test)

In [ ]:
X_new.shape

In [ ]:
arr = []
for i in range(1,33):
 arr.append('PC{}'.format(i))

In [ ]:
arr

In [ ]:
print(pd.DataFrame(pca.components_,columns=X.columns,index = arr))


In [ ]:
test_new = pd.DataFrame(test_new,columns = arr)

In [ ]:
train_new = pd.DataFrame(X_new,columns = arr)

In [ ]:
train_new.to_csv('train_pca.csv',index=False)
test_new.to_csv('test_pca.csv',index=False)

In [ ]:
!cp -r '/content/train_pca.csv' /content/drive/MyDrive/project9/PCA
!cp -r '/content/test_pca.csv' /content/drive/MyDrive/project9/PCA


In [ ]:

N_SPLITS = 5
N_REPEATS = 3
EARLY_STOPPING_ROUNDS = 300
VERBOSE = False
SEED = 17171

In [ ]:
target_column = 'label'

In [ ]:
features = [c for c in train_new.columns if c not in [target_column]]

In [ ]:
len(features)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X = train_new
y=train['label']
X.shape, y.shape

In [ ]:
## going by the optuna way:
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
# Which hyperparameters to tune: https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

def objective(trial):
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 101,2)
    min_samples_split = trial.suggest_float("min_samples_split", 0, 1)
    n_estimators = trial.suggest_int("n_estimators", 1, 999,2)
    max_depth = trial.suggest_int("max_depth", 1, 101,2)
    max_features = trial.suggest_categorical("max_features",["auto","sqrt"])  
    rfc = RandomForestClassifier(min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_estimators=n_estimators, max_depth = max_depth, max_features = max_features)
        
    rskfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)
    score = cross_val_score(rfc, X,y, scoring='accuracy', cv=rskfold, n_jobs = -1,error_score='raise')
    score = score.mean()
    return score

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50, timeout = 4*60*60)

In [ ]:
trial = study.best_trial      
print('Accuracy: {}'.format(trial.value))   

In [ ]:
print("Best hyperparameters: {}".format(trial.params))

In [ ]:
rfc_params  = {'min_samples_leaf': 1, 'min_samples_split': 0.0024272599847029556, 'n_estimators': 769, 'max_depth': 33, 'max_features': 'auto'}


In [ ]:

from sklearn.metrics import recall_score,precision_score,f1_score
from sklearn.metrics import classification_report
import statistics
from statistics import mean

In [ ]:
%%time
N_SPLITS = 5
y_preds = []
acc= []
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits = N_SPLITS)

for fold, (train_id, valid_id) in enumerate(folds.split(X,y)):
  X_train = X.iloc[train_id]
  y_train = y.iloc[train_id]
  X_valid = X.iloc[valid_id]
  y_valid = y.iloc[valid_id]
  model = RandomForestClassifier(**rfc_params)
  
  model.fit(X_train, y_train)
  valid_pred = model.predict(X_valid)

#print('fold:{}'.format(fold+1))
  #print('Validation Accuracy score : {}'.format(accuracy_score(y_valid, valid_pred)))
  #print('-'*100)
  acc.append(accuracy_score(y_valid, valid_pred))

  y_preds.append(model.predict(test_new))


In [ ]:
print('the mean accuracy is: {}'.format(mean(acc)))


In [ ]:
y_preds

In [ ]:
from collections import Counter
def mode(data):
    """Return the most common data point from discrete or nominal data.
    ``mode`` assumes discrete data, and returns a single value. This is the
    standard treatment of the mode as commonly taught in schools:
        >>> mode([1, 1, 2, 3, 3, 3, 3, 4])
        3
    This also works with nominal (non-numeric) data:
        >>> mode(["red", "blue", "blue", "red", "green", "red", "red"])
        'red'
    If there are multiple modes with same frequency, return the first one
    encountered:
        >>> mode(['red', 'red', 'green', 'blue', 'blue'])
        'red'
    If *data* is empty, ``mode``, raises StatisticsError.
    """
    pairs = Counter(iter(data)).most_common(1)
    try:
        return pairs[0][0]
    except IndexError:
        raise StatisticsError('no mode for empty data') from None




In [ ]:
arr = [y_preds[0][0],y_preds[1][0], y_preds[2][0], y_preds[3][0], y_preds[4][0]]

In [ ]:
y_pred = pd.DataFrame(columns=['rfc_pca'])
pred= []
for j in range(0,100):
    arr = [y_preds[0][j], y_preds[1][j], y_preds[2][j], y_preds[3][j],y_preds[4][j]]
    pred.append(mode(arr))

In [ ]:
y_pred["rfc_pca"]=pred

In [ ]:
y_pred.to_csv('rfc_pca_pred.csv', index=False)

In [ ]:
!cp -r '/content/rfc_pca_pred.csv' /content/drive/MyDrive/project9/PCA



In [ ]:
y_pred